In [271]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import html
import re
import json
import time

In [2]:
#структура сайта
main_url = "http://marspb.ru"
types = ['tram', 'trolleybus', 'bus', 'marshrutka']

In [214]:
#scrapper
def getRoutsHrefFromPage(page_href):
    url = main_url + page_href
    
    r = requests.get(url)
    text = r.text
    soup = BeautifulSoup(text, "lxml")
    
    rout_items = soup.find_all('div', {'class': 'route-item'})
    hrefs = list(map(lambda rout: rout.find('a').get('href'), rout_items))
    numbers = list(map(lambda rout: rout.find('a').text, rout_items))
    
    routs = []
    for i in range(0, len(rout_items)):
        rout = {'href':hrefs[i], 'number': numbers[i]}
        routs.append(rout)
        
    return routs

def getRoutsHrefs():
    routs = []
    
    for mar_type in types:
        href = "/" + mar_type + "/"
         
        routs_part = getRoutsHrefFromPage(href)
        wtf = list(map(lambda rout: rout.update({'type':mar_type}), routs_part))
        
        routs.extend(routs_part)
        
        #Необходимо обходить страницы
        url = main_url + href
        r = requests.get(url)
        text = r.text
        soup = BeautifulSoup(text, "lxml")
        
        page_wrapper = soup.find('div', {'class': 'pager-wrapper'})
        if page_wrapper:
            pages_href = page_wrapper.find_all('a')
            pages = list(map(lambda x : x.get('href'), pages_href))
            
            for page in pages:
                routs_part = getRoutsHrefFromPage(page)
                wtf = list(map(lambda rout: rout.update({'type':mar_type}), routs_part))
                routs.extend(routs_part)
    
    return(routs)

def getRout(rout):
    url = main_url + rout['href']
    r = requests.get(url)
    text = r.text
    soup = BeautifulSoup(text, 'lxml')
    
    rout_text = soup.find_all('script')[-1].text
    rout_text = re.sub("^\s+|\n|\r|\t|\s+$", '', rout_text)
    rout_text = re.sub('.*Route: ', '', rout_text)
    rout_text = re.sub(',MapIconsDirectory.*', '', rout_text)
    
    rout_dict = json.loads(rout_text)
    rout_pf = rout_dict['pf'] if 'pf' in rout_dict else {}
    rout_pb = rout_dict['pb'] if 'pb' in rout_dict else {}
    
    rout.update({'pf':rout_pf, 'pb': rout_pb})
    
    return rout
    

def getAllRouts():    
    routs = getRoutsHrefs()
    for rout in routs:
        getRout(rout)
        time.sleep(1)
        
        
    return routs

In [215]:
routs = getAllRouts()

In [265]:
#convert to geojson
geojson = {
    "type": "FeatureCollection",
    
    "features": [
    {
        "type": "Feature",
        "properties": {
            "href" : r["href"],
            "number" : r["number"],
            "direction": r["direction"]
        },
        "geometry" : {
            "type": "LineString",
            "coordinates": [[rc["lng"], rc["lat"]] for rc in r['rout']],
            },
        
     } for r in routs]
}

In [226]:
rout_fd = list(map(lambda r: {"href":r["href"], "number": r["number"], "direction": "f", "rout": r['pf']}, routs))

In [227]:
rout_pd = list(map(lambda r: {"href":r["href"], "number": r["number"], "direction": "b", "rout": r['pb']}, routs))

In [230]:
routs = []
routs.extend(rout_fd)
routs.extend(rout_pd)

In [264]:
type(geojson['features'][1]['geometry']['coordinates'][0])

set

In [272]:
output = open("public_transport_routs.geojson", 'w')
json.dump(geojson, output)

In [267]:
type(geojson)

dict

In [269]:
geojson['features'][-2]

{'geometry': {'coordinates': [[30.697351, 59.784345],
   [30.696921, 59.784297],
   [30.696483, 59.784417],
   [30.695187, 59.784888],
   [30.692261, 59.785813],
   [30.690914, 59.786172],
   [30.690005, 59.786357],
   [30.6882, 59.786632],
   [30.685897, 59.786948],
   [30.685034, 59.787159],
   [30.684652, 59.787279],
   [30.684313, 59.7875],
   [30.682735, 59.788763],
   [30.681128, 59.79007],
   [30.679782, 59.791111],
   [30.678932, 59.791759],
   [30.678449, 59.792075],
   [30.677881, 59.792366],
   [30.675155, 59.793348],
   [30.672124, 59.794399],
   [30.667567, 59.795985],
   [30.665972, 59.796597],
   [30.664488, 59.797081],
   [30.663378, 59.797409],
   [30.657029, 59.79879],
   [30.64972, 59.800368],
   [30.647587, 59.800788],
   [30.646922, 59.800893],
   [30.646433, 59.800942],
   [30.645844, 59.800973],
   [30.637303, 59.80081],
   [30.636778, 59.800799],
   [30.636359, 59.800821],
   [30.63593, 59.800859],
   [30.635384, 59.800948],
   [30.634838, 59.801074],
   [30.633